## Load Dependencies

In [24]:
import matplotlib.pyplot as plt # for plotting
from tmaze_toolkit.data.extraction import selectDoorCoords, extractDoorTraces, initial_coords
from tmaze_toolkit.processing.signal import bandpass_filter, process_door_traces
from tmaze_toolkit.visualization.plotDoorTraces import plotDoorTraces
from tmaze_toolkit.data.openFunctions import openDoorTracesPkl
from tmaze_toolkit.processing.extractTrialTimes import extract_floor_traces, extract_trial_times, verify_correct_trial_times
from tmaze_toolkit.data.jsonProcessing import load_json_files, add_trajectories, save_outDict
from tmaze_toolkit.visualization.plotTrajectory import plot_trajectory
from tmaze_toolkit.processing.normalize import normalize_trajectory

In [25]:
pkl_file = r"N:\TMAZE\TMAZE_REFIND_VID_NEW\Cropped_ATO1_Vidoes\ATO1_2025-03-22T18_28_44_doorTraces.pkl"
dat = openDoorTracesPkl(pkl_file)
dat = process_door_traces(dat)
events = extract_trial_times(dat, pad_frames=90)

Expected 346 trial starts, but got 357
Expected 94 trial starts, but got 90
Expected 94 trial ends, but got 90
94
94
Trial 0 likely has a missed detection in the doors 1 and 2
Trial 1 likely has a missed detection in the doors 1 and 2
Trial 2 likely has a missed detection in the doors 1 and 2
Trial 3 likely has a missed detection in the doors 1 and 2
Trial 4 likely has a missed detection in the doors 1 and 2
Trial 5 likely has a missed detection in the doors 1 and 2
Trial 6 likely has a missed detection in the doors 1 and 2
Trial 7 likely has a missed detection in the doors 1 and 2
Trial 8 likely has a missed detection in the doors 1 and 2
Trial 9 likely has a missed detection in the doors 1 and 2
Trial 10 likely has a missed detection in the doors 1 and 2
Trial 11 likely has a missed detection in the doors 1 and 2
Trial 12 likely has a missed detection in the doors 1 and 2
Trial 13 likely has a missed detection in the doors 1 and 2
Trial 14 likely has a missed detection in the doors 1

In [26]:
jsonFileLocation = r"N:\TMAZE\TMAZE_DATA\data_ATO1_20250322*"
verify_correct_trial_times(events, jsonFileLocation)

Found 2 json files
Animal ID: ATO1
Working on file N:\TMAZE\TMAZE_DATA\data_ATO1_20250322_182807.json
Working on file N:\TMAZE\TMAZE_DATA\data_ATO1_20250322_190937.json
Trial times verified successfully


True

In [27]:
outDict = load_json_files(jsonFileLocation)



Found 2 json files
Animal ID: ATO1
Working on file N:\TMAZE\TMAZE_DATA\data_ATO1_20250322_182807.json
Working on file N:\TMAZE\TMAZE_DATA\data_ATO1_20250322_190937.json


In [28]:
print(outDict[89])

{'MazeTexture': 'CNO', 'rewardBefLick': False, 'lickUntilCorrect': False, 'decision': 'incorrect', 'floorID': 'F,4', 'time_for_trial': 1, 'valve': 4, 'lick': 'left(V1)', 'trial_number': 40, 'trial_time': [2025, 3, 22, 19, 47, 0, 5, 81, 1], 'right_probability': 0.5, 'rewardAmount(ms)': 5, 'BiasCorrection(T/F)': False, 'coneHeight_1(floor1)': '0.5', 'coneHeight_2(floor2)': '1', 'Injection': '1'}


In [29]:
from tmaze_toolkit.data.deeplabcutProcessing import load_deeplabcut_files
dlc_file = r"N:\TMAZE\TMAZE_REFIND_VID_NEW\Cropped_ATO1_Vidoes\ATO1_2025-03-22T18_28_44DLC_resnet50_Intermidiate textured DLC model Sep10shuffle1_100000.h5"
dlc_dict = load_deeplabcut_files(dlc_file)

In [30]:
outDict = add_trajectories(outDict, dlc_dict, events)


In [31]:
print(outDict[3]['lick'])

right(V2)


In [32]:
save_outDict(outDict, jsonFileLocation)

In [33]:
import cv2
import numpy as np
import os

def select_corners(videoFile):
    # Open video and grab middle frame
    vid = cv2.VideoCapture(videoFile)
    if not vid.isOpened():
        print(f"Error: Could not open video {videoFile} for corner selection.")
        return None
    
    length = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    middle_frame_pos = min(int(length/2), length - 1) if length > 0 else 0
    vid.set(cv2.CAP_PROP_POS_FRAMES, middle_frame_pos)
    ret, frame = vid.read()
    if not ret:
        print(f"Error: Could not read frame from {videoFile}.")
        vid.release()
        return None
    
    # Create window for corner selection
    window_name = "Select 4 corners (click in order, press ENTER when done)"
    cv2.namedWindow(window_name)
    
    # Store coordinates and current corner
    corners = {}
    current_corner = 1
    
    # Mouse callback function
    def mouse_callback(event, x, y, flags, param):
        nonlocal current_corner, corners, frame, display_img
        
        if event == cv2.EVENT_LBUTTONDOWN and current_corner <= 4:
            # Add corner to the dictionary
            corners[f"corner{current_corner}"] = (x, y)
            print(f"Corner {current_corner} selected at ({x}, {y})")
            current_corner += 1
            
            # Update display image
            display_img = frame.copy()
            for i in range(1, min(current_corner, 5)):
                cx, cy = corners[f"corner{i}"]
                cv2.circle(display_img, (cx, cy), 5, (0, 0, 255), -1)  # Red circle
                cv2.putText(display_img, f"{i}", (cx+10, cy), cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (0, 0, 255), 2)
    
    # Set mouse callback
    cv2.setMouseCallback(window_name, mouse_callback)
    
    # Make a copy for displaying
    display_img = frame.copy()
    
    # Main loop
    while True:
        cv2.imshow(window_name, display_img)
        key = cv2.waitKey(1) & 0xFF
        
        # Press Enter to finish when all 4 corners are selected
        if (key == 13 and current_corner > 4) or key == 27:  # Enter or Esc
            break
    
    cv2.destroyAllWindows()
    vid.release()
    
    # Check if all 4 corners were selected
    if current_corner <= 4:
        print(f"Warning: Only {current_corner-1} corners were selected. Need 4 corners.")
    
    return corners
    



In [34]:
videoFile = r"N:\TMAZE\TMAZE_REFIND_VID_NEW\Cropped_ATO1_Vidoes\ATO1_2025-03-22T18_28_44.mp4"

In [35]:
def drawLine(point1, point2):
    """
    write the equation of the line between two points
    """
    x1, y1 = point1
    x2, y2 = point2
    slope = (y2 - y1) / (x2 - x1)
    intercept = y1 - slope * x1
    return [slope, intercept]


In [36]:
def calculateIntersection(line1, line2):
    """
    Calculate the intersection of two lines
    """
    slope1, intercept1 = line1
    slope2, intercept2 = line2
    x = (intercept2 - intercept1) / (slope1 - slope2)   
    y = slope1 * x + intercept1
    return [x, y]



In [37]:
def calculate_length(point1, point2):
    """
    Calculate the length of the line between two points
    """
    x1, y1 = point1
    x2, y2 = point2
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)


In [38]:
def normalize_points(x, y, corners):
    """
    Normalize the points to the corners of the rectangle
    Input:
        x: x coordinates of the points
        y: y coordinates of the points
        corners: dictionary of the corners
    Output:
        x: normalized x coordinates
        y: normalized y coordinates
    """
    input_point = (x, y)

    point1 = corners['corner1']
    point2 = corners['corner2']
    point3 = corners['corner3']
    point4 = corners['corner4']


    #Draw a line between points 1 and 3
    topline = drawLine(point1, point3)
    
    bottomline = drawLine(point2, point4)
    
    rightline = drawLine(point1, point2)
   
    leftline = drawLine(point3, point4)
   
    # Calculate intersection between topline and bottomline
    intersectionY = calculateIntersection(topline, bottomline)
  
    intersectionX = calculateIntersection(rightline, leftline)
    

    # Draw a line between intersectionY and input_point
    yLine = drawLine(intersectionY, input_point)
   
    # Calculate intersection between intersectionX and input_point
    xLine = drawLine(intersectionX, input_point)
    

    # find intersection between yLine and rightline
    intersectionYwithRightline = calculateIntersection(yLine, rightline)
    

    # find intersection between xLine and topline
    intersectionXwithTopline = calculateIntersection(xLine, topline)


    toplineLength = calculate_length(point1, point3)
    bottomlineLength = calculate_length(point2, point4)
    rightlineLength = calculate_length(point1, point2)
    leftlineLength = calculate_length(point3, point4)

    # Find the ratio between intersectionXwithTopline and entire length of topline
    ratioX = intersectionXwithTopline[0] / toplineLength
    ratioY = intersectionYwithRightline[1] / rightlineLength


    return ratioX, ratioY


In [39]:
def input_numpy_array(x, y, corners):
    """
    Input:
        x: x coordinates of the points
        y: y coordinates of the points
        corners: dictionary of the corners
    """
    x_new = np.copy(x)
    y_new = np.copy(y)
    for i in range(len(x)):
        x_new[i], y_new[i] = normalize_points(x[i], y[i], corners)
    return x_new, y_new

In [40]:
print(outDict[0])

{'MazeTexture': 'CNO', 'rewardBefLick': False, 'lickUntilCorrect': False, 'decision': 'incorrect', 'floorID': 'F,4', 'time_for_trial': 2, 'valve': 4, 'lick': 'left(V1)', 'trial_number': 1, 'trial_time': [2025, 3, 22, 18, 28, 23, 5, 81, 1], 'right_probability': 0.5, 'rewardAmount(ms)': 5, 'BiasCorrection(T/F)': False, 'coneHeight_1(floor1)': '0.5', 'coneHeight_2(floor2)': '1', 'Injection': '1', 'trajectory': scorer    DLC_resnet50_Intermidiate textured DLC model Sep10shuffle1_100000  \
bodyparts                                                          RightEye   
coords                                                                    x   
145                                                16.396782                  
146                                                16.170279                  
147                                                15.625341                  
148                                                15.320689                  
149                                 

In [41]:
def normalize_trajectory(outDict, videoFile):
    corners = select_corners(videoFile)
    if not corners:
        print("Failed to get corners, cannot normalize")
        return
    body_parts = outDict[0]['trajectory'].columns.get_level_values('bodyparts').unique()
    scorer = outDict[0]['trajectory'].columns.get_level_values('scorer')[0]
    for trial_id in range(0, len(outDict) - 1):
        dipshit = outDict[trial_id]['trajectory'].copy()
    
    return dipshit
    

In [42]:
def normalize_trajectory(outDict, videoFile):
    # Get corners from video
    corners = select_corners(videoFile)
    if not corners:
        print("Failed to get corners, cannot normalize")
        return
    
    # Get the DLC scorer name 
    scorer = outDict[0]['trajectory'].columns.get_level_values('scorer')[0]
     # Get all body parts
    body_parts = outDict[0]['trajectory'].columns.get_level_values('bodyparts').unique()
    # For each trial in the dictionary
    for trial_id in range(0, len(outDict) - 1):
        # Create a copy for the optimized trajectory
        outDict[trial_id]['trajectoryOptomized'] = outDict[trial_id]['trajectory'].copy()
        # Process each body part
        for body_part in body_parts:
            x_original = np.copy(outDict[trial_id]['trajectory'][scorer, body_part, 'x'].values)
            y_original = np.copy(outDict[trial_id]['trajectory'][scorer, body_part, 'y'].values)
            #x, y = input_numpy_array(x, y, corners)
           
            x_transformed, y_transformed = input_numpy_array(x_original, y_original, corners)

            """
            # Update with transformed coordinates
            outDict[trial_id]['trajectoryOptomized'].loc[:, (scorer, body_part, 'x')] = x  # Replace with transformed x
            outDict[trial_id]['trajectoryOptomized'].loc[:, (scorer, body_part, 'y')] = y  # Replace with transformed y
            """

            outDict[trial_id]['trajectoryOptomized'][(scorer, body_part, 'x')] = x_transformed  # Replace with transformed x
            outDict[trial_id]['trajectoryOptomized'][(scorer, body_part, 'y')] = y_transformed  # Replace with transformed y
    return outDict

In [43]:
print(outDict.keys())

dict_keys(['originalFiles', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89])


In [44]:
print(normalize_trajectory(outDict, videoFile))



Corner 1 selected at (8, 10)
Corner 2 selected at (12, 193)
Corner 3 selected at (255, 2)
Corner 4 selected at (257, 197)
{'originalFiles': ['N:\\TMAZE\\TMAZE_DATA\\data_ATO1_20250322_182807.json', 'N:\\TMAZE\\TMAZE_DATA\\data_ATO1_20250322_190937.json'], 0: {'MazeTexture': 'CNO', 'rewardBefLick': False, 'lickUntilCorrect': False, 'decision': 'incorrect', 'floorID': 'F,4', 'time_for_trial': 2, 'valve': 4, 'lick': 'left(V1)', 'trial_number': 1, 'trial_time': [2025, 3, 22, 18, 28, 23, 5, 81, 1], 'right_probability': 0.5, 'rewardAmount(ms)': 5, 'BiasCorrection(T/F)': False, 'coneHeight_1(floor1)': '0.5', 'coneHeight_2(floor2)': '1', 'Injection': '1', 'trajectory': scorer    DLC_resnet50_Intermidiate textured DLC model Sep10shuffle1_100000  \
bodyparts                                                          RightEye   
coords                                                                    x   
145                                                16.396782                  
146           

In [45]:
outDict[0]['trajectory'].droplevel(0, axis=1)
outDict[0]['trajectoryOptomized'].droplevel(0, axis=1)

bodyparts  RightEye                        LeftEye                       \
coords            x         y likelihood         x         y likelihood   
145        0.063981  0.203077   0.204299  0.071595  0.205665   0.214569   
146        0.063048  0.204067   0.263299  0.070187  0.206538   0.275817   
147        0.060755  0.209398   0.487721  0.066394  0.210290   0.476064   
148        0.059541  0.208116   0.519488  0.064783  0.209233   0.514393   
149        0.056547  0.205028   0.572518  0.061932  0.206695   0.577590   
...             ...       ...        ...       ...       ...        ...   
1001       0.568020  0.990500   0.043443  0.569276  0.972801   0.042462   
1002       0.567237  0.998600   0.064903  0.574160  0.973489   0.067431   
1003       0.506273  0.979357   0.102078  0.534395  0.986056   0.036524   
1004       0.506885  0.973344   0.083399  0.550992  0.976229   0.074442   
1005       0.508211  0.984066   0.195226  0.513026  0.970731   0.163578   

bodyparts  RightEar                        LeftEar  ...            UpperSpine  \
coords            x         y likelihood         x  ... likelihood          x   
145        0.061434  0.158287   0.073063  0.066815  ...   0.064447   0.056570   
146        0.045139  0.176755   0.082051  0.066800  ...   0.072162   0.064823   
147        0.067303  0.190002   0.132967  0.070060  ...   0.109494   0.066271   
148        0.066146  0.189467   0.137273  0.069035  ...   0.113905   0.065649   
149        0.064383  0.187744   0.161881  0.068261  ...   0.129019   0.064687   
...             ...       ...        ...       ...  ...        ...        ...   
1001       0.579835  1.004018   0.286693  0.573496  ...   0.388917   0.560968   
1002       0.569594  0.998812   0.615314  0.584432  ...   0.673840   0.563105   
1003       0.548398  1.011267   0.360539  0.545669  ...   0.297293   0.540759   
1004       0.546806  0.994764   0.726655  0.543445  ...   0.801529   0.536595   
1005       0.511410  0.985366   0.577172  0.538455  ...   0.634647   0.518809   

bodyparts                      MiddleSpine                       TailBase  \
coords            y likelihood           x         y likelihood         x   
145        0.186447   0.947796    0.060567  0.156323   0.983391  0.082755   
146        0.179449   0.950220    0.060677  0.156551   0.981634  0.081976   
147        0.179368   0.951037    0.059174  0.160131   0.975342  0.080878   
148        0.178306   0.947172    0.059321  0.160651   0.972662  0.080898   
149        0.176189   0.936252    0.060202  0.159463   0.970914  0.081015   
...             ...        ...         ...       ...        ...       ...   
1001       0.972701   0.707375    0.549998  0.973593   0.601273  0.501164   
1002       0.977772   0.929617    0.549970  0.981192   0.827195  0.498936   
1003       0.975377   0.690354    0.530910  0.974595   0.599544  0.467603   
1004       0.968925   0.971073    0.516396  0.964651   0.949933  0.459521   
1005       0.971254   0.933702    0.500701  0.966738   0.972645  0.447171   

bodyparts                       
coords            y likelihood  
145        0.115154   0.992139  
146        0.117729   0.990714  
147        0.125322   0.985815  
148        0.126352   0.984501  
149        0.125757   0.982633  
...             ...        ...  
1001       0.972917   0.590049  
1002       0.978000   0.784409  
1003       0.981734   0.534536  
1004       0.966960   0.942590  
1005       0.964155   0.952556  

[861 rows x 21 columns]

In [46]:
from tmaze_toolkit.visualization.plotTrajectory import plot_normalized_trajectory
plot_normalized_trajectory(outDict, videoFile)


NameError: name 'select_corners' is not defined